# Ordering


## Warning: This notebook will place live orders

Use a paper trading account (during market hours).


In [1]:
from ib_insync import *
from pprint import pprint
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4008, clientId=13)
# util.logToConsole()

API connection failed: ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 4008)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 10061] Connect call failed ('127.0.0.1', 4008)

Create a contract and a market order:

In [22]:
contract = Stock(symbol='PSTG', exchange='SMART', currency='USD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 10, 40)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [23]:
trade = ib.placeOrder(contract, order)

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [24]:
ib.sleep(1)
pprint(trade.log)

[TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 7, 956247, tzinfo=datetime.timezone.utc),
               status='PendingSubmit',
               message='',
               errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 8, 23303, tzinfo=datetime.timezone.utc),
               status='PreSubmitted',
               message='',
               errorCode=0)]


Let's modify the limit price and resubmit:

In [25]:
order.lmtPrice = 35.35

ib.placeOrder(contract, order)

Trade(contract=Stock(conId=208813725, symbol='PSTG', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='PSTG', tradingClass='PSTG'),
      order=LimitOrder(orderId=10, clientId=13, permId=642390569, action='SELL', totalQuantity=10.0, lmtPrice=35.35, auxPrice=0.0),
      orderStatus=OrderStatus(orderId=10,
                              status='PreSubmitted',
                              filled=0.0,
                              remaining=10.0,
                              avgFillPrice=0.0,
                              permId=642390569,
                              parentId=0,
                              lastFillPrice=0.0,
                              clientId=13,
                              whyHeld='',
                              mktCapPrice=0.0),
      fills=[],
      log=[TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 7, 956247, tzinfo=datetime.timezone.utc),
                         status='PendingSubmit',
                         message=''

And now cancel it:

In [26]:
ib.cancelOrder(order)

Trade(contract=Stock(conId=208813725, symbol='PSTG', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='PSTG', tradingClass='PSTG'), order=LimitOrder(orderId=10, clientId=13, permId=642390569, action='SELL', totalQuantity=10.0, lmtPrice=35.35, auxPrice=0.0), orderStatus=OrderStatus(orderId=10, status='PendingCancel', filled=0.0, remaining=10.0, avgFillPrice=0.0, permId=642390569, parentId=0, lastFillPrice=0.0, clientId=13, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 7, 956247, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 8, 23303, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 28, 282979, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Modify', errorCode=0), TradeLogEntry(time=datetime.datetime(2022, 10, 11, 

In [27]:
pprint(trade.log)

[TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 7, 956247, tzinfo=datetime.timezone.utc),
               status='PendingSubmit',
               message='',
               errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 8, 23303, tzinfo=datetime.timezone.utc),
               status='PreSubmitted',
               message='',
               errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 10, 11, 22, 59, 28, 282979, tzinfo=datetime.timezone.utc),
               status='PreSubmitted',
               message='Modify',
               errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 10, 11, 23, 0, 5, 797462, tzinfo=datetime.timezone.utc),
               status='PendingCancel',
               message='',
               errorCode=0),
 TradeLogEntry(time=datetime.datetime(2022, 10, 11, 23, 0, 5, 878741, tzinfo=datetime.timezone.utc),
               status='Cancelled',
               message='',
               errorCode=0)]


# What are our positions?

In [29]:
pprint(ib.positions())

[Position(account='DU2926038', contract=Stock(conId=2585323, symbol='MTB', exchange='NYSE', currency='USD', localSymbol='MTB', tradingClass='MTB'), position=4.0, avgCost=156.991525),
 Position(account='DU2926038', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=20000.0, avgCost=1.074),
 Position(account='DU2926038', contract=Stock(conId=6459, symbol='DIS', exchange='NYSE', currency='USD', localSymbol='DIS', tradingClass='DIS'), position=100.0, avgCost=102.11),
 Position(account='DU2926038', contract=Stock(conId=118089500, symbol='ABBV', exchange='NYSE', currency='USD', localSymbol='ABBV', tradingClass='ABBV'), position=-100.0, avgCost=118.488668),
 Position(account='DU2926038', contract=Stock(conId=2009113, symbol='SPG', exchange='NYSE', currency='USD', localSymbol='SPG', tradingClass='SPG'), position=100.0, avgCost=146.06)]


# Stream tick data

In [30]:
ib.reqMktData(contract)
ticker = ib.ticker(contract)
ib.sleep(2)
pprint(ticker)

Ticker(contract=Stock(conId=208813725, symbol='PSTG', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='PSTG', tradingClass='PSTG'), time=datetime.datetime(2022, 10, 11, 23, 0, 36, 202779, tzinfo=datetime.timezone.utc), bid=26.4, bidSize=1.0, ask=27.75, askSize=1.0, last=27.03, lastSize=0.0, volume=3637709.0, open=27.0, high=27.52, low=26.4, close=27.3, halted=0.0, ticks=[TickData(time=datetime.datetime(2022, 10, 11, 23, 0, 36, 202779, tzinfo=datetime.timezone.utc), tickType=1, price=26.4, size=1.0), TickData(time=datetime.datetime(2022, 10, 11, 23, 0, 36, 202779, tzinfo=datetime.timezone.utc), tickType=2, price=27.75, size=1.0), TickData(time=datetime.datetime(2022, 10, 11, 23, 0, 36, 202779, tzinfo=datetime.timezone.utc), tickType=4, price=27.03, size=0.0), TickData(time=datetime.datetime(2022, 10, 11, 23, 0, 36, 202779, tzinfo=datetime.timezone.utc), tickType=5, price=27.03, size=0.0), TickData(time=datetime.datetime(2022, 10, 11, 23, 0, 36, 202779, tzinfo=datet

In [31]:
ticker.marketPrice()

27.03

In [32]:
ib.sleep(1)
ticker.marketPrice()

27.03

# Historical Data

In [33]:
pprint(ib.reqHistoricalData(
        contract,
        endDateTime="",
        durationStr="10 D",
        barSizeSetting="1 min",
        whatToShow="MIDPOINT",
        useRTH=True,
        formatDate=1,
    ))

Error 162, reqId 12: Historical Market Data Service error message:No market data permissions for NYSE STK. Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details., contract: Stock(conId=208813725, symbol='PSTG', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='PSTG', tradingClass='PSTG')


[]


In [34]:
ib.disconnect()